In [ ]:
# Fine Tuning Legal Bert Model

In [ ]:

!pip install transformers pandas datasets torch scikit-learn
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import datasets
import torch
import re
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import Dataset


In [ ]:
from huggingface_hub import login
tok = ''
login(token=tok)


In [ ]:
# Collect and merge multiple csv,excel files into single dataframe
def combine_multiple_files(files_dir_path):
  combined_files = pd.DataFrame()
  for files in os.listdir(files_dir_path):
    if files.endswith('.tsv'):
      f_path = os.path.join(files_dir_path,files)
      df = pd.read_csv(f_path,sep='\t')
      combined_files = pd.concat([combined_files,df])
    elif files.endswith('.csv'):
      f_path = os.path.join(files_dir_path,files)
      df = pd.read_csv(f_path,sep=',')
      combined_files = pd.concat([combined_files,df])
    elif files.endswith('.xlsx'):
      f_path = os.path.join(files_dir_path,files)
      df = pd.read_excel(f_path)
      combined_files = pd.concat([combined_files,df])
  return combined_files




In [ ]:
def merge_multiple_dataframe(dataframe_list,merge_by=0):
  df = pd.concat(dataframe_list, ignore_index=True)
  return df

In [ ]:
def clean_text(text):
    # Remove leading and trailing spaces from the text
    text = text.strip()

    # Ensure only one space exists between sentences
    text = re.sub(r'\s*\.\s*', '. ', text)  # Handle spaces around periods
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces in general

    # Ensure no space before the first sentence and after the last sentence
    return text.strip()

In [ ]:
def remove_varicon_pattern(text):
    # Regular expression to match '(varicon <number>)' at the end of the string
    cleaned_text = re.sub(r'\s*\(variation \d+\)\s*$', '', text)
    return cleaned_text

In [ ]:
# REQUIRED FIELDS
required_fields = ["Fill Form","Approval",'Notify']

In [ ]:

sheet_name = 'Task type identification'
new_data_syn = '/content/Balanced_Workflow_Task_Dataset_modified_syn.csv'
# data = pd.read_excel(data_path,sheet_name=sheet_name)
# data = pd.read_csv(new_data_syn)
# data

In [ ]:
# load all the datasets
# load_complex_datasets_path = '/content/sample_data/complex_tsv/'

# data = combine_multiple_files(load_complex_datasets_path)
# data.columns = ['Text','labels']
# data.reset_index(inplace=True,drop=True)
# data

In [ ]:
# def save_model_pytorch(output_dir = "/content/finetuned_model"):
#   os.makedirs(output_dir, exist_ok=True)
#   # Save model weights
#   torch.save(model.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))
#   print('Successfully Saved Model in "pytorch_model.bin"')

# save model
def save_model_and_tokenizer(model, tokenizer, output_dir="/content/finetuned_model"):
    os.makedirs(output_dir, exist_ok=True)

    # Save the model's architecture and weights (state_dict)
    torch.save(model.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))

    # Save the tokenizer (vocab, config, etc.)
    tokenizer.save_pretrained(output_dir)

    # Save the model configuration (optional but recommended)
    model.config.save_pretrained(output_dir)

    print(f"Successfully saved model and tokenizer in {output_dir}")


# reload model for inference

def load_model_and_tokenizer(model_path="/content/finetuned_model"):
    # Load the model configuration (optional but recommended)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)

    # Load the model's weights (state_dict)
    model.load_state_dict(torch.load(os.path.join(model_path, "pytorch_model.bin")))

    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Ensure the model is on the correct device
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

    print(f"Successfully loaded model and tokenizer from {model_path}")

    return model, tokenizer



In [ ]:
df_fill_form = pd.read_csv('/content/fill_form_dataset.tsv',sep='\t')
df_fill_form.reset_index(drop=True,inplace=True)
# rename columns name
df_fill_form.columns = ['Text','labels']

df_notify = pd.read_csv('/content/notify_dataset.tsv',sep='\t')
df_notify.reset_index(drop=True,inplace=True)
# rename columns name
df_notify.columns = ['Text','labels']

df_approval = pd.read_csv('/content/approval_dataset.tsv',sep='\t')
df_approval.reset_index(drop=True,inplace=True)
df_approval.columns = ['Text','labels']

balanced_dataset = pd.read_csv('/content/Balanced_Workflow_Task_Dataset_modified_syn.csv')
balanced_dataset = balanced_dataset[['Text','Task Type']]
balanced_dataset.rename(columns={'Task Type':'labels'},inplace=True)
balanced_dataset = balanced_dataset.query('labels in @required_fields')
balanced_dataset

,Text,labels
0,"To start the project scoping process, the requ...",Approval
1,I need to get final signoff from Project Manag...,Approval
2,I need to get final signoff from Project Manag...,Approval
3,Once all relevant stakeholders have provided t...,Approval
4,"As part of the compliance process, I will subm...",Approval
...,...,...
1495,ML manager reviews the documentation on train...,Notify
1496,Documentation related to the system architectu...,Notify
1497,"High risk system once identified, the result i...",Notify
1498,Upon the AI Manager's approval of the risk mea...,Notify


In [ ]:
data = merge_multiple_dataframe([df_fill_form,df_notify,df_approval,balanced_dataset])
data

,Text,labels
0,"According to Article 18 of the EU AI Act, the ...",Fill Form
1,"According to Article 13 of the EU AI Act, the ...",Fill Form
2,"According to Article 5 of the EU AI Act, any E...",Fill Form
3,"According to Article 24 of the EU AI Act, the ...",Fill Form
4,"According to Article 7 of the EU AI Act, any E...",Fill Form
...,...,...
16492,ML manager reviews the documentation on train...,Notify
16493,Documentation related to the system architectu...,Notify
16494,"High risk system once identified, the result i...",Notify
16495,Upon the AI Manager's approval of the risk mea...,Notify


In [ ]:
data = data.dropna()
data

,Text,labels
0,"According to Article 18 of the EU AI Act, the ...",Fill Form
1,"According to Article 13 of the EU AI Act, the ...",Fill Form
2,"According to Article 5 of the EU AI Act, any E...",Fill Form
3,"According to Article 24 of the EU AI Act, the ...",Fill Form
4,"According to Article 7 of the EU AI Act, any E...",Fill Form
...,...,...
16492,ML manager reviews the documentation on train...,Notify
16493,Documentation related to the system architectu...,Notify
16494,"High risk system once identified, the result i...",Notify
16495,Upon the AI Manager's approval of the risk mea...,Notify


In [ ]:
data['Text'] = data['Text'].apply(remove_varicon_pattern) # remove_varicon_pattern
data['Text'] = data['Text'].apply(clean_text)
data

,Text,labels
0,"According to Article 18 of the EU AI Act, the ...",Fill Form
1,"According to Article 13 of the EU AI Act, the ...",Fill Form
2,"According to Article 5 of the EU AI Act, any E...",Fill Form
3,"According to Article 24 of the EU AI Act, the ...",Fill Form
4,"According to Article 7 of the EU AI Act, any E...",Fill Form
...,...,...
16492,ML manager reviews the documentation on traini...,Notify
16493,Documentation related to the system architectu...,Notify
16494,"High risk system once identified, the result i...",Notify
16495,Upon the AI Manager's approval of the risk mea...,Notify


In [ ]:
data.duplicated().sum()

1405

In [ ]:
data.drop_duplicates(inplace=True)
data

,Text,labels
0,"According to Article 18 of the EU AI Act, the ...",Fill Form
1,"According to Article 13 of the EU AI Act, the ...",Fill Form
2,"According to Article 5 of the EU AI Act, any E...",Fill Form
3,"According to Article 24 of the EU AI Act, the ...",Fill Form
4,"According to Article 7 of the EU AI Act, any E...",Fill Form
...,...,...
16009,ML manager reviews the documentation on traini...,Notify
16010,Documentation related to the system architectu...,Notify
16011,"High risk system once identified, the result i...",Notify
16012,Upon the AI Manager's approval of the risk mea...,Notify


In [ ]:
set(data['labels'].values)

{'Approval', 'Fill Form', 'Notify'}

In [ ]:
# Alternatively, to see the frequency of each unique value
value_counts = data['labels'].value_counts()
print("\nFrequency of each unique value in 'Task Type':")
print(value_counts)


Frequency of each unique value in 'Task Type':
labels
Approval     5043
Fill Form    5033
Notify       5016
Name: count, dtype: int64


In [ ]:
# required_fields = ['Fill_Form','Notify']
data = data.query('labels in @required_fields')
data.reset_index(inplace=True, drop=True)
data

,Text,labels
0,"According to Article 18 of the EU AI Act, the ...",Fill Form
1,"According to Article 13 of the EU AI Act, the ...",Fill Form
2,"According to Article 5 of the EU AI Act, any E...",Fill Form
3,"According to Article 24 of the EU AI Act, the ...",Fill Form
4,"According to Article 7 of the EU AI Act, any E...",Fill Form
...,...,...
15087,ML manager reviews the documentation on traini...,Notify
15088,Documentation related to the system architectu...,Notify
15089,"High risk system once identified, the result i...",Notify
15090,Upon the AI Manager's approval of the risk mea...,Notify


In [ ]:
n_labels = len(value_counts.values)
n_labels

3

In [ ]:
n_percent_validate_data = 20
n_val_data = int(len(data) * n_percent_validate_data/100)
n_val_data

3018

In [ ]:
# random_rows = data.copy().sample(n=3)
# # Reset the index for cleaner output
validate_data = data.copy().groupby('labels').apply(lambda x: x.sample(n=int(n_val_data/n_labels))).reset_index(drop=True)

validate_data

<ipython-input-25-8f5da25904bd>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  validate_data = data.copy().groupby('labels').apply(lambda x: x.sample(n=int(n_val_data/n_labels))).reset_index(drop=True)


,Text,labels
0,"According to Article 5 of the EU AI Act, the C...",Approval
1,"According to Article 9 of the EU AI Act, the F...",Approval
2,"According to Article 15 of the EU AI Act, the ...",Approval
3,"According to Article 12 of the EU AI Act, the ...",Approval
4,"According to Article 19 of the EU AI Act, the ...",Approval
...,...,...
3013,"According to Article 6 of the EU AI Act, pleas...",Notify
3014,"According to Article 20 of the EU AI Act, plea...",Notify
3015,"According to Article 4 of the EU AI Act, pleas...",Notify
3016,"According to Article 19 of the EU AI Act, plea...",Notify


In [ ]:
# Convert each row to a tuple and use sets to filter
set_large = set(tuple(x) for x in data.to_records(index=False))
set_small = set(tuple(x) for x in validate_data.to_records(index=False))

# Find the difference between the two sets (rows in df_large but not in df_small)
filtered_set = set_large - set_small

# Convert the result back to a DataFrame
train_data = pd.DataFrame(filtered_set, columns=data.columns)
train_data

,Text,labels
0,"According to Article 6 of the EU AI Act, the C...",Approval
1,"According to Article 14 of the EU AI Act, plea...",Notify
2,"According to Article 9 of the EU AI Act, the D...",Fill Form
3,"According to Article 26 of the EU AI Act, the ...",Approval
4,"According to Article 12 of the EU AI Act, plea...",Notify
...,...,...
12069,"According to Article 25 of the EU AI Act, the ...",Fill Form
12070,"According to Article 11 of the EU AI Act, plea...",Notify
12071,"According to Article 4 of the EU AI Act, pleas...",Notify
12072,"According to Article 26 of the EU AI Act, plea...",Notify


In [ ]:
validate_data = validate_data.sample(frac=1).reset_index(drop=True)
validate_data

,Text,labels
0,"According to Article 22 of the EU AI Act, plea...",Notify
1,"According to Article 24 of the EU AI Act, plea...",Notify
2,"According to Article 24 of the EU AI Act, the ...",Fill Form
3,"According to Article 27 of the EU AI Act, the ...",Approval
4,"According to Article 10 of the EU AI Act, plea...",Notify
...,...,...
3013,"According to Article 18 of the EU AI Act, the ...",Approval
3014,"According to Article 24 of the EU AI Act, the ...",Approval
3015,"According to Article 26 of the EU AI Act, the ...",Approval
3016,"According to Article 16 of the EU AI Act, the ...",Approval


In [ ]:
# Define the label mapping
label_mapping = {"Fill Form": 0, "Approval": 1, "Notify":2}
# label_mapping = {"Form": 0, "Notiify": 1}
# Apply the mapping to the 'label' column
train_data["labels"] = train_data["labels"].replace(label_mapping)
validate_data["labels"] = validate_data["labels"].replace(label_mapping)

<ipython-input-28-53aaad845727>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data["labels"] = train_data["labels"].replace(label_mapping)
<ipython-input-28-53aaad845727>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  validate_data["labels"] = validate_data["labels"].replace(label_mapping)


In [ ]:
validate_data

,Text,labels
0,"According to Article 22 of the EU AI Act, plea...",2
1,"According to Article 24 of the EU AI Act, plea...",2
2,"According to Article 24 of the EU AI Act, the ...",0
3,"According to Article 27 of the EU AI Act, the ...",1
4,"According to Article 10 of the EU AI Act, plea...",2
...,...,...
3013,"According to Article 18 of the EU AI Act, the ...",1
3014,"According to Article 24 of the EU AI Act, the ...",1
3015,"According to Article 26 of the EU AI Act, the ...",1
3016,"According to Article 16 of the EU AI Act, the ...",1


In [ ]:
data.shape

(15092, 2)

In [ ]:
lr_rate = 5e-5
num_epochs = 3
model_name = 'Non_extra_custom_layer_fine_tuned_model_lr_'
model_version = model_name + str(lr_rate) + '_' + str(num_epochs) + '_' + str(data.shape[0])
model_version

'Non_extra_custom_layer_fine_tuned_model_lr_5e-05_3_15092'

In [ ]:


# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(validate_data)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
def preprocess_function(data_):
    return tokenizer(data_["Text"], truncation=True, padding="max_length", max_length=128)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

# Model
model = BertForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=len(label_mapping))

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_dir="./logs",  # Directory to save logs
    logging_strategy="steps",  # Log training loss periodically
    logging_steps=1,  # Log training loss every 10 steps
    learning_rate=lr_rate, # 2e-3
    per_device_train_batch_size=16,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to=[],  # Disable wandb or other logging integrations
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test
)

# Fine-tune the model
trainer.train()




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Map:   0%|          | 0/3018 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.000100,0.006159
2,0.000100,0.010484
3,0.000100,0.011025


TrainOutput(global_step=2265, training_loss=0.013097179847278978, metrics={'train_runtime': 845.701, 'train_samples_per_second': 42.831, 'train_steps_per_second': 2.678, 'total_flos': 2382623554235904.0, 'train_loss': 0.013097179847278978, 'epoch': 3.0})

In [ ]:
save_model_and_tokenizer(model, tokenizer, output_dir = f"./{model_version}")

Successfully saved model and tokenizer in ./Non_extra_custom_layer_fine_tuned_model_lr_5e-05_3_15092


In [ ]:
# DOWNLOAD FINETUNED BERT MODEL

def download_finetuned_model(dir_prefix='/content/', model_dir_name=None):
    from google.colab import files
    if model_dir_name is None:
        raise ValueError('Model version name is NoneType')

    # Construct the full path to the directory
    absolute_dir_path = os.path.join(dir_prefix, model_dir_name)
    if not os.path.exists(absolute_dir_path):
        raise FileNotFoundError(f"The directory {absolute_dir_path} does not exist.")

    # Compress the directory into a zip file
    zip_file_name = f"{model_dir_name}.zip"
    zip_command = f"zip -r {zip_file_name} {model_dir_name}"
    os.system(zip_command)

    # Verify if the zip file was created successfully
    if not os.path.exists(zip_file_name):
        raise FileNotFoundError(f"Failed to create the zip file: {zip_file_name}")

    # Download the compressed file
    files.download(zip_file_name)
    print("Download Complete")

download_finetuned_model(model_dir_name=model_version)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download Complete


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# we can use pipeline for Inference
path = f'/content/{model_version}'
# Load the classification pipeline
classifier = pipeline("text-classification", model=path, tokenizer=path)

# Example input text
text = 'start the project scoping process by filling out the required form with all the necessary details. Also need to capture the necessary details about the project in a form, ensuring everything aligns with the intended objectives and regulatory requirements. In the next step I need to  get final signoff from Project Manager with comments on all potential concerns are addressed. Eventually, all stakholders within this process will be sent letter with final updates.'
mapping_label = {'LABEL_2':'Notify','LABEL_0':'Fill Form','LABEL_1':'Approval'}
k = 'information .'
# Get probabilities for all class labels
results = classifier(k, top_k=None)
map_results = {mapping_label[result['label']]:result['score'] for result in results}
# Display results
# print(results)
print(map_results)


Device set to use cuda:0


{'Fill Form': 0.4609777629375458, 'Approval': 0.2721886932849884, 'Notify': 0.2668335735797882}


In [1]:
# test data validation
sheet_test = 'New data 25th Dec'
new_test_data = pd.read_excel('/content/new_test_finplus_data.xlsx',sheet_name = sheet_test)
new_test_data = new_test_data[['Text','Task Type']]
new_test_data.rename(columns={'Task Type':'labels'},inplace=True)
new_test_data

In [ ]:
# balanced dataset
path = '/content/Balanced_Workflow_Task_Dataset.csv'
new_test_data = pd.read_csv(path)
new_test_data = new_test_data[['Text','Task Type']]
new_test_data.rename(columns={'Task Type':'labels'},inplace=True)
new_test_data

In [ ]:
# drop null data
new_test_data = new_test_data.dropna()
new_test_data

In [ ]:
replacement_rules = {
    "Fillform": "Fill Form",
}
new_test_data["labels"] = new_test_data["labels"].replace(replacement_rules, regex=True)
new_test_data

In [ ]:
# Filter Test Labels
test_fields = ['Approval','Fill Form','Notify']
new_test_data = new_test_data.query('labels in @test_fields')
new_test_data

In [ ]:
new_test_data['Text'] = new_test_data['Text'].apply(remove_varicon_pattern) # remove_varicon_pattern
new_test_data['Text'] = new_test_data['Text'].apply(clean_text)
new_test_data

In [ ]:
original_label = []
predicted_label = []
pred_prob = []
org_text = []

mapping_label = {'LABEL_2':'Notify','LABEL_0':'Fill Form','LABEL_1':'Approval'}

for data in  new_test_data.iterrows():
  text = data[1]['Text']

  label = data[1]['labels']
  original_label.append(label)
  org_text.append(text)

  # Get probabilities for all class labels
  results = classifier(text, top_k=None)
  map_results = {mapping_label[result['label']]:result['score'] for result in results}

  high_prob_label = max(map_results, key=map_results.get)
  predicted_label.append(high_prob_label)
  pred_prob.append(map_results[high_prob_label])

In [ ]:
df = pd.DataFrame({'Text':org_text,'Original_Label':original_label,'Predicted_Label':predicted_label,'Probability_Score':pred_prob})
df

In [ ]:
well_predicted = (df["Original_Label"] == df["Predicted_Label"]).sum()
well_predicted, len(df)